In [1]:
import pandas as pd 
import csv

In [4]:
pip install pyspark 

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pytest 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.6/343.6 kB 6.8 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, ArrayType
import pytest
# import mysql.connector

In [9]:
def create_session():
    spark = SparkSession.builder.appName("Venkata").config("spark.driver.extraClassPath","./mysql-connector-java-5.1.49.jar").getOrCreate()
    sqlcontext = SQLContext(spark)
    return spark, sqlcontext
spark, sqlcontext = create_session()
df = spark.read.csv('./testing_data.csv', header=True)
df.show()

+-----------------+--------------------+------------------+--------------+--------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------+-------------------+---------------+------------+
|        Full Name|      Street Address|              City|         State|Zip Code|             Country|        Phone Number|               Email|Date of Birth|          Occupation|        Company Name|        Job Position|         Website URL|     Company Address|Preferred Language| Profile Picture URL|Marital Status|Social Media Handle|Company Revenue|Last Updated|
+-----------------+--------------------+------------------+--------------+--------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+----------------

In [4]:
schema = df.schema
print(schema)
prefix = None
fields = []
datatype = []
for field in schema.fields:
    print(field)
    name = prefix + '.' + field.name if prefix else field.name
    dtype = field.dataType
    datatype.append(dtype)
    print(datatype)
    if isinstance (dtype, StructType):
        name = prefix + '.' + field.name if prefix else field.name
        print(name)
    else:
        fields.append(name)

    print(fields)

StructType([StructField('Full Name', StringType(), True), StructField('Street Address', StringType(), True), StructField('City', StringType(), True), StructField('State', StringType(), True), StructField('Zip Code', StringType(), True), StructField('Country', StringType(), True), StructField('Phone Number', StringType(), True), StructField('Email', StringType(), True), StructField('Date of Birth', StringType(), True), StructField('Occupation', StringType(), True), StructField('Company Name', StringType(), True), StructField('Job Position', StringType(), True), StructField('Website URL', StringType(), True), StructField('Company Address', StringType(), True), StructField('Preferred Language', StringType(), True), StructField('Profile Picture URL', StringType(), True), StructField('Marital Status', StringType(), True), StructField('Social Media Handle', StringType(), True), StructField('Company Revenue', StringType(), True), StructField('Last Updated', StringType(), True)])
StructField('

In [6]:
ddl = df._jdf.schema().toDDL().replace("'","").replace(",",",\n").replace(" ","_").replace("_STRING"," STRING")
generate_ddl_statement = f"CREATE TABLE ddl_statement (\n {ddl});"
print(generate_ddl_statement)

CREATE TABLE ddl_statement (
 `Full_Name` STRING,
`Street_Address` STRING,
City STRING,
State STRING,
`Zip_Code` STRING,
Country STRING,
`Phone_Number` STRING,
Email STRING,
`Date_of_Birth` STRING,
Occupation STRING,
`Company_Name` STRING,
`Job_Position` STRING,
`Website_URL` STRING,
`Company_Address` STRING,
`Preferred_Language` STRING,
`Profile_Picture_URL` STRING,
`Marital_Status` STRING,
`Social_Media_Handle` STRING,
`Company_Revenue` STRING,
`Last_Updated` STRING);


In [11]:
df.write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/data_frame?useSSL=false&allowPublicKeyRetrieval=true") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "venkat_copy") \
    .option("user", "venkata") \
    .option("password", "Root!123$$") \
    .mode("append") \
    .save()